# Automação Web (selenium)

- Projeto de automação web que iráautomatizar pesquisa de preços com base em uma planilha de produtos, para esse projeto iremos utilizar a biblioteca Selenium do python.

# Entendimento do negócio

- Um funcionário trabalha na área de compras de uma empresa e precisa fazer uma comparação de fornecedores para os seus insumos/produtos.

- Esse funcionário constantemente busca nos sites desses fornecedores os produtos disponíveis e o preço, afinal, cada um deles pode fazer promoção em momentos diferentes e com valores diferentes.

# Objetivo

- Seu objetivo: Criar uma pesquisa automática que, irá buscar por produtos com valor abaixo de mercado, então deve-se criar uma planilha com esses produtos, indicando o preço, nome do produto e link para compra.

- Em seguida, vai enviar um e-mail com a lista destes produtos para o setor de compra.

- OBS: No nosso caso, vamos fazer com produtos comuns em sites como Google Shopping e Buscapé, mas a ideia é a mesma para outros sites. Essa busca também pode ser feita por APIs.


### O que temos disponível?

- Planilha de Produtos, com os nomes dos produtos, o preço máximo, o preço mínimo (para evitar produtos "errados" ou "baratos de mais para ser verdade" e os termos que vamos querer evitar nas nossas buscas.


# Importando bibliotecas

In [71]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import pandas as pd
import win32com.client as win32

# Coleta de dados (PC)

In [72]:
# tabela de filtros desejados
tabela_prod = pd.read_excel("buscas.xlsx")
display(tabela_prod)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64 gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500


# Coleta de dados (WEB)

#### Funcão de busca no google:

In [73]:
def busca_google_shopping(nav, produto, termos_banidos, preco_min, preco_max):

    # abre o navegador
    nav.get("https://www.google.com.br/")

    # tratar os valores da busca
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    preco_max = float(preco_max)
    preco_min = float(preco_min)

    # criando lista de termos banidos (split remove espacos e cria lista com nomes)
    lista_term_banidos = termos_banidos.split(' ')
    lista_term_prod = produto.split(' ')


    # Vai no input e digita o produto
    nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input").send_keys(produto) 
    # dar um enter
    nav.find_element(By.XPATH, "/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input").send_keys(Keys.ENTER) 
    # clicar em shopping: para shopping vamos pegar todos elementos
    # da classe "hdtb-mitem" e procurar por "shopping" usando "elementS"
    elementos = nav.find_elements(By.CLASS_NAME, "hdtb-mitem")
    for item in elementos:
        if "Shopping" in item.text:
            item.click()
            break #paramos pois pode ter mais de um item

    # pegar preços, nome e link, pra isso vamos buscar dentro de uma
    # div que contém todos esses elementos

    # elemento div
    lista_resultado = nav.find_elements(By.CLASS_NAME, "sh-dgr__grid-result")

    # lista que será preenchida com resultados
    lista_ofertas = []
    
    # dentro de cada div procura os elementos
    for resultado in lista_resultado:
        nome = resultado.find_element(By.CLASS_NAME, 'tAxDx').text
        # so deixando em minusculo para comparação
        nome = nome.lower()

        #criar um filtro para os termos banidos
        tem_termos_banidos = False
        for palavra in lista_term_banidos:
            if palavra in nome:
                tem_termos_banidos = True  

        tem_todos_termos_produto = True
        for palavra in lista_term_prod:
            if palavra not in nome:
                tem_todos_termos_produto = False

        if tem_termos_banidos == False and tem_todos_termos_produto == True:
            try: #tratando caso preço venha com formato errado (ex: )
                # se tem_termos_banidos = false e tem_todos _termos_produto = true continua
                # tratar o preço para comparação
                preco = resultado.find_element(By.CLASS_NAME, 'a8Pemb').text
                preco = preco.replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.')
                if preco != "":
                    preco = float(preco)
                  
                # verificar se o preço está na faixa:
                if preco_min <= preco <= preco_max:      
                    # pelo modo abaixo nao deu certo pegar o link, vamos para outra forma
                    # link = resultado.find_element(By.CLASS_NAME, 'shntl').get_attribute('href')
                    # entao vamos pegar o link pelo PARENT de outro elemento
                    elemento_link = resultado.find_element(By.CLASS_NAME,'aULzUe')
                    # XPATH ".." é o elemento pai que está acima do filho
                    elemento_pai = elemento_link.find_element(By.XPATH,'..')
                    link = elemento_pai.get_attribute('href')
                    #print(nome, preco, link)
                    #vams criar uma lista com tuplas dos resultados
                    lista_ofertas.append((nome, preco, link))
            except: 
                continue

 
    return lista_ofertas




#### Função de busca no buscapé:

In [74]:
def busca_buscape(nav, produto, termos_banidos, preco_min, preco_max):

    # tratar os valores da busca
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()

    # criando lista de termos banidos (split remove espacos e cria lista com nomes)
    lista_term_banidos = termos_banidos.split(' ')
    lista_term_prod = produto.split(' ')

    # Abre o site buscapé
    nav.get('https://www.buscape.com.br/?og=19220&og=19220&gclid=Cj0KCQiA5NSdBhDfARIsALzs2EAHF8kmSAOhiehyzmAm-5A5huHec6Fm998lSNeRllTOaK3sfrIZV4gaAtBjEALw_wcB')
    # Digita o produto na busca
    nav.find_element(By.CLASS_NAME,'AutoCompleteStyle_input__HG105').send_keys(produto)
    # Enter
    nav.find_element(By.CLASS_NAME,'AutoCompleteStyle_input__HG105').send_keys(Keys.ENTER)
    # time para carregamento da página
    # OBS: muito cuidado com time, pois pegará dados erroneos caso a página
    # nao tiver uma espera no carregamento
    time.sleep(5)

    elementos = nav.find_elements(By.CLASS_NAME,'SearchCard_ProductCard_Inner__7JhKb')

    # lista que será preenchida com resultados
    lista_ofertas = []
    
    for elemento in elementos:    
        nome = elemento.find_element(By.CLASS_NAME, 'Text_Text__h_AF6').text
        nome = nome.lower()
        
        #criar um filtro para os termos banidos
        tem_termos_banidos = False
        for palavra in lista_term_banidos:
            if palavra in nome:
                tem_termos_banidos = True  

        tem_todos_termos_produto = True
        for palavra in lista_term_prod:
            if palavra not in nome:
                tem_todos_termos_produto = False

        if tem_termos_banidos == False and tem_todos_termos_produto == True:
            try:
                # se tem_termos_banidos = false e tem_todos _termos_produto = true continua
                # tratar o preço para comparação
                preco = elemento.find_element(By.CLASS_NAME,'Text_MobileHeadingS__Zxam2').text
                preco = preco.replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.')
                if preco != "":
                    preco = float(preco)
                    
                # verificar se o preço está na faixa:
                if preco_min <= preco <= preco_max:      
                    link = elemento.get_attribute('href')
                    #vams criar uma lista com tuplas dos resultados
                    lista_ofertas.append((nome, preco, link)) 
            except: 
                continue

            
    return lista_ofertas

#### Passa os parametros e realiza as buscas:

In [75]:
# Abre o navegador
nav = webdriver.Chrome()

#criação de tabela
tabela_geral = pd.DataFrame()

for linha in tabela_prod.index:
    
    # dados da busca
    produto = tabela_prod.loc[linha, 'Nome']
    termos_banidos = tabela_prod.loc[linha, 'Termos banidos']
    preco_min = tabela_prod.loc[linha, 'Preço mínimo']
    preco_max = tabela_prod.loc[linha, 'Preço máximo']
    

    lista_ofertas_GS = busca_google_shopping(nav, produto, termos_banidos, preco_min, preco_max)
    if lista_ofertas_GS: #existe item na tabela?
        tabela_ofertas_GS = pd.DataFrame(lista_ofertas_GS, columns=['nome', 'preco', 'link'])
        # Se tabela de ofertas tem itens então adiciona a tabela geral
        tabela_geral = pd.concat([tabela_geral, tabela_ofertas_GS])

        
    else:
        #se tabela vazia
        tabela_ofertas_GS = None
        

    lista_ofertas_buscape = busca_buscape(nav, produto, termos_banidos, preco_min, preco_max)
    if lista_ofertas_buscape:
        tabela_ofertas_buscape = pd.DataFrame(lista_ofertas_buscape, columns=['nome', 'preco', 'link'])
        tabela_geral = pd.concat([tabela_geral, tabela_ofertas_buscape])
    else:
        # se tabela vazia
        tabela_ofertas_buscape = None
    break    
        
display(tabela_geral)




,nome,preco,link
0,iphone 12 64gb - azul - estou zerado,3329.10,https://www.google.com.br/url?url=https://www....
1,iphone 12 64gb - preto - estou zerado,3329.10,https://www.google.com.br/url?url=https://www....
2,iphone 12 preto 64gb,3000.00,https://www.google.com.br/url?url=https://rj.o...
3,"iphone 11 apple (64gb) branco, tela de 6,1, 4g...",3491.03,https://www.google.com.br/url?url=https://www....


#### Exportando as bases de dados para excel:

In [76]:
# os index estão fora de ordem, pois pegou de tabelas diferentes, vamos resetar:
# drop=True descarta os indices antigos
tabela_geral = tabela_geral.reset_index(drop=True)
# salvando para excel: index=False não salva coluna index
tabela_geral.to_excel("Pesquisa geral.xlsx", index=False)



#### Enviando email para compras:

In [77]:
# Verifica se encontrou alguma oferta na tabela
# caso contrario não encia o email
if len(tabela_geral.index) > 0:

    outlook = win32.Dispatch('outlook.application')

    mail = outlook.CreateItem(0)
    mail.To = 'dev.daniel.amorim@gmail.com'
    mail.Subject = 'Pesquisa de preços'
    mail.HTMLBody = f'''
    Bom dia,

    Segue em anexo a pesquisa de preços!
    {tabela_geral.to_html(index=False)}

    '''

    # Anexos (pode ser quantos quiser):
    attachment = r"C:\Users\devda\Documents\GitHub\Data Science\DS-Automacao_web\Pesquisa geral.xlsx"
    mail.Attachments.Add(attachment)
    # Envia o email
    mail.Send()
    
# Ao final encerra o navegador    
nav.quit()

# FIM!
#### Desafio, refazer o mesmo projeto usando APIs do google shopping e do buscapé